<a href="https://colab.research.google.com/github/saikoushiknalubola/anndata_annam/blob/main/Challenge-1./notebooks/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

soil_classification_path = kagglehub.competition_download('soil-classification')

print('Data source import complete.')


In [ ]:
"""
Author: Annam.ai IIT Ropar
Team Name: anndata
Team Members: N. Saikoushik, M. Sai Teja, G. Navya Sri, N. Chandhana Priya, V. Asmitha
Leaderboard Rank: 22

⚠️ NOTE: This inference notebook uses untrained ImageNet weights and random mapping to soil classes.
It is only for testing the pipeline; it will not give meaningful predictions.
"""

import os
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data paths
data_dir = "/kaggle/input/soil-classification/soil_classification-2025"
test_dir = os.path.join(data_dir, "test")
test_ids_file = os.path.join(data_dir, "test_ids.csv")

# Load test ids
df_test = pd.read_csv(test_ids_file)

# Define soil class labels (static, as no training was done)
class_labels = ['Alluvial', 'Black', 'Clay', 'Red']
num_classes = len(class_labels)

# Load pretrained ResNet18 (ImageNet weights)
model_cnn = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model_cnn.fc = torch.nn.Linear(model_cnn.fc.in_features, num_classes)

# Randomly initialize last layer (no fine-tuning)
torch.nn.init.xavier_uniform_(model_cnn.fc.weight)

# Send model to device
model_cnn = model_cnn.to(device)
model_cnn.eval()

# Image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Predict function
def predict_image(path):
    img = Image.open(path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model_cnn(img_tensor)
        _, predicted = torch.max(outputs, 1)
    return class_labels[predicted.item()]

# Run inference on test set
predictions = []
for img_id in tqdm(df_test['image_id'], desc="Predicting"):
    img_path = os.path.join(test_dir, img_id)
    pred_label = predict_image(img_path)
    predictions.append(pred_label)

# Prepare submission
df_submission = pd.DataFrame({
    'image_id': df_test['image_id'],
    'soil_type': predictions
})

submission_path = "submission.csv"
df_submission.to_csv(submission_path, index=False)
print(f"✅ Submission saved to {submission_path}")


Predicting: 100%|██████████| 341/341 [00:21<00:00, 15.97it/s]

✅ Submission saved to submission.csv
